In [100]:
import pandas as pd
import numpy as np
import psycopg2

# db connect 
conn = psycopg2.connect( 
    host = "localhost", 
    dbname = "naver_blog", 
    user = "postgres", 
    password = "1234"
    )         
sslmode = "require"
print("Connection established")

cursor = conn.cursor()

# Fetch all rows from table
cursor.execute("SELECT 내용 FROM 시린이;")
result = cursor.fetchall()

# 데이터프레임으로 만들기
시린이_df = pd.DataFrame(result)
시린이_df.columns = [i[0] for i in  cursor.description]
시린이_df.head(10)
# Cleanup
# conn.commit()
# cursor.close()
# conn.close()


Connection established


,내용
0,언젠가부터 왼쪽 어금니 사이에 \n음식물이 끼기 시작했는데요.\n양치를 할 때마다 ...
1,마스크 시대가 열리면서\n저의 입냄새를 경험하게 되었어요\n 제가 맡기에도 고약했고...
2,날이 갑자기 더워지면서\n마스크안에서 숨쉬는 게\n답답해지더라구요.\n일할 때도 덥...
3,전 양치할때 이를 빡빡 닦는 편이에요. \n이게 잘못된 습관이라는건 알고 있지만\n...
4,마스크를 착용하면서 어쩔 수 없이\n제 자신의 입냄새를 맡게 되었는데요.\n가끔은 ...
5,오랜시간 마스크를 착용하면서\n깨닫게 된 한가지가 바로 저의 입냄새에요.\n가끔은 ...
6,"안녕하세요\n오늘은 ""시린이 치약""에대해 소개해드리려 합니다^^\n날씨가 풀리면서 ..."
7,최근에 온도가 27도까지 올랐었죠.\n초봄인데도 벌써 날씨가 이러는 걸 보니\n앞으...
8,마스크를 쓰고 다니니까\n입에서 냄새가 나는 게 확연하게\n느껴지고 불쾌하기까지...
9,특별히 차가운 음식을 먹지 않았는데도\n치아의 시림이 발생하더라고요.\n이가 많이 ...


In [101]:
시린이_df['내용'].duplicated().sum()

0

In [ ]:
시린이_df['내용'].replace('', np.nan, inplace=True) # np.nan으로 결측값을 인식할 때 해결
시린이_df

In [103]:
def nomalize(df):
    df_copy = df.copy() # 데이터프레임 원본 카피
    df_isnull = df_copy.isnull().sum().sum() # 전체 결측 데이터 합산
    print('결측치의 전체합 : ', df_isnull) #  결측치 합산값 확인
    # df_fill = df_copy.fillna(0)
    # df_drop = df_fill.dropna(axis=0)
    df_drop = df_copy.dropna(axis=0)
    return df_drop

a_df = nomalize(시린이_df)
a_df


결측치의 전체합 :  0


,내용
0,언젠가부터 왼쪽 어금니 사이에 \n음식물이 끼기 시작했는데요.\n양치를 할 때마다 ...
1,마스크 시대가 열리면서\n저의 입냄새를 경험하게 되었어요\n 제가 맡기에도 고약했고...
2,날이 갑자기 더워지면서\n마스크안에서 숨쉬는 게\n답답해지더라구요.\n일할 때도 덥...
3,전 양치할때 이를 빡빡 닦는 편이에요. \n이게 잘못된 습관이라는건 알고 있지만\n...
4,마스크를 착용하면서 어쩔 수 없이\n제 자신의 입냄새를 맡게 되었는데요.\n가끔은 ...
...,...
95,https://blog.naver.com/talove0831/222056441644...
96,수원치과 치아마모로 시린이 \n날도 추운데 치아도 추워요\n물한잔 편안하게 마시지 ...
97,이번에 사용해본 치약은\n투비짱 \n시린이 잇몸 치약 \n입냄새제거 불소치약 \n으...
98,구매후기만 쓰는 배당노마드입니다.\n시린이 확실히 완화되었음\n센소다인 치약쓰기전까...


In [104]:
# 데이터프레임 csv파일로 저장
시린이_df.to_csv('df_시린이_내용.csv', index=False, header=True, encoding='utf-8-sig') # encoding='utf-8-sig' : 크롤링 후 csv에 저장할 때

### 자연어처리

In [3]:
# 원본 파일 복사하기
c잇몸영양제_df = 잇몸영양제_df.copy()
c잇몸영양제_df.head()

,내용
0,양치질을 아무리 잘한다고 하더라도\n잘못된 양치질 또는 잇몸 자체가 약한 경우에는\...
1,아무리 하루 세 번 양치를 잘한다 해도\n치아에는 쉽게 문제가 생기곤 하는데...
2,나이가 드니 치아도 약해지나 봐요.\n옛날이면 돌도 씹어 먹었을 텐데\n이제는 그토...
3,잇몸 영양제 조금 아쉬웠던 이유\n한번씩 잇몸이 욱신대는 느낌이 느껴질때가 있어요....
4,어머니께서 예전부터 잇몸이 좋지 않아\n최근에 임플란트를 예정 중에 계신데 치과에서...


In [4]:
import numpy as np
import nltk
import matplotlib.pyplot as plt
import re # 정규표현식
import urllib.request
from tqdm import tqdm # 작업 진행률 표시
from tensorflow.keras.preprocessing.text import Tokenizer # 텍스트 토큰화
from tensorflow.keras.preprocessing.sequence import pad_sequences # 길이가 다른 여러 문장의 길이를 임의로 동일하게 맞춰줌

In [20]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from collections import Counter
from konlpy.tag import Okt

In [19]:
# 자연어처리 라이브러리 불러오기
okt = Okt()

In [5]:
# 한글과 공백 제외 모두 제거 : 띄어쓰기 유지, 구두점 제거

c잇몸영양제_df['내용'] = c잇몸영양제_df['내용'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]',"")
c잇몸영양제_df[:10]

C:\Users\User\AppData\Local\Temp/ipykernel_1372/1458016269.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  c잇몸영양제_df['내용'] = c잇몸영양제_df['내용'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]',"")


,내용
0,양치질을 아무리 잘한다고 하더라도잘못된 양치질 또는 잇몸 자체가 약한 경우에는잇몸 ...
1,아무리 하루 세 번 양치를 잘한다 해도치아에는 쉽게 문제가 생기곤 하는데얼마...
2,나이가 드니 치아도 약해지나 봐요옛날이면 돌도 씹어 먹었을 텐데이제는 그토록 좋아하...
3,잇몸 영양제 조금 아쉬웠던 이유한번씩 잇몸이 욱신대는 느낌이 느껴질때가 있어요그런 ...
4,어머니께서 예전부터 잇몸이 좋지 않아최근에 임플란트를 예정 중에 계신데 치과에서 상...
5,안녕하세요 치과입니다저희 병원에 치주 질환이 심해져서 내원해 주시는환자분들을 살펴보...
6,저도 이제 연식이 어느정도 되다보니 요즘엔 질긴 고기나 딱딱한 음식을 먹을 때면 치...
7,엄마가 인플란트를 준비하고 계신데 치과에서 잇몸상태가 좋지 않아서 치료를 조금 더 ...
8,안녕하세요 밤곰이에요 치아 건강은 오복 중 하나라는 말이 있어요 그만큼 치아는 우리...
9,세까지 함께하는 치아와 잇몸년 전쯤 오른쪽 아래 어금니가 너무 아파서 참고 참다 치...


In [ ]:
#  PyKoSpacing : 띄어쓰기 임의로 만들기
# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [6]:
from pykospacing import Spacing

# 데이터프레임 특정 열값을 리스트로 변환
spacing_cen = []
for sc in c잇몸영양제_df['내용']:
    spacing_cen.append(sc)
# print(spacing_cen)

# 리스트 문자열 변환
result = ' '.join(s for s in spacing_cen)
# print(result)

In [7]:
# 띄어쓰기가 없는 문장 임의로 만들기
new_sent = result.replace(" ", '') 
# print(new_sent)

In [8]:
# 띄어쓰기
spacing = Spacing()
kospacing_sent = spacing(result) 
# print(sent) # 원본 확인용   
# print(kospacing_list)

In [11]:
# 문법 정규화
kospacing_list = [okt.normalize(kospacing_sent)]

In [ ]:
# Py_Hanspell : 맞춤법 검사_데이터가 크면 안 됨
# !pip install git+https://github.com/ssut/py-hanspell.git
# from hanspell import spell_checker
# spelled_sent = spell_checker.check(kospacing_sent)

# hanspell_sent = spelled_sent.checked
# print(hanspell_sent)

In [16]:
import os

os.chdir(r"C:\Users\User\anaconda3\Lib\site-packages\konlpy\java")
os.getcwd()
os.makedirs('./my_dict')

!cp C:\\Users\\User\\anaconda3\\Lib\\site-packages\\konlpy\\java\\open-korean-text-2.1.0.jar C:\\Users\\User\\anaconda3\\Lib\\site-packages\\konlpy\\java\\my_dict\\open-korean-text-2.1.0.jar
!jar xvf C:\\Users\\User\\anaconda3\\Lib\\site-packages\\konlpy\\java\\my_dict\\open-korean-text-2.1.0.jar

'cp'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'jar'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [ ]:
# 사용자 사전 열기
with open(f"/usr/local/lib/python3.7/dist-packages/konlpy/java/org/openkoreantext/processor/util/noun/names.txt") as f:
    data = f.read()
# 새로운 단어 추가
data += '존잘키링연하남\n제현이\n치과\n상황극\n전동\n면도기\n'
# 파일 새롭게 저장
with open("/usr/local/lib/python3.7/dist-packages/konlpy/java/org/openkoreantext/processor/util/noun/names.txt", 'w') as f:
    f.write(data)

!jar cvf /usr/local/lib/python3.7/dist-packages/konlpy/java/aaa/open-korean-text-2.1.0.jar * 
# 확인용
a = spell_checker.check('존잘키링연하남')
result = a.checked
print(result)

okt.morphs('존잘키링연하남')


In [9]:
# 불용어 제거 

bul_word = []
stopwords = ['같이','많은','특히','때문','그런','처럼','또한','의','가','이','은','들','을','하는','로','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','것','못','된','또는','에는','에서','그리고','많이','대해','있지만','다시','또','이렇게','함께','해','경우','저','수','할','라고','때']

stop_words = set(stopwords)

for word_tokens in kospacing_list:
    bul_word.append([word for word in okt.morphs(word_tokens) if not word in stop_words])

# print(bul_word)

# 이중리스트 제거
blist = sum(bul_word, [])

NameError: name 'kospacing_list' is not defined

In [ ]:
# 명사 추출

n_list=[]
for word in blist:
    n_list.append(okt.nouns(word))

# 이중리스트 제거
final_n = sum(n_list, [])
print(final_n)

In [41]:
# 리스트를 딕셔너리로 생성하고 빈도수에 따라 키워드를 정렬하는 함수
def count_f(my_list):
    count = {} # 딕셔너리 생성

    for item in my_list:
        count[item] = count.get(item, 0) + 1
    
    final_df = (pd.DataFrame(list(count.items()),
                            columns=['keyword', 'count']))
    
    # 빈도수가 10보다 작다면 해당 행 삭제
    d_row = final_df[final_df['count'] < 20].index
    final_df = final_df.drop(d_row)

    # count 칼럼에 따라서 내림차순 정렬
    final_df = final_df.sort_values(by=['count', 'keyword'], ascending=False)
    final_df = final_df.reset_index(drop=True) # index 초기화    
    
    return final_df

# 데이터프레임 csv파일로 저장하는 함수
def df_export_csv(my_df):
    my_file = input('저장할 파일의 이름을 입력하세요 : ')
    exp = my_df.to_csv(('{0}.csv').format(my_file), index=False, header=False, encoding='cp949')
    
    return exp


a = count_f(final_n)
b = df_export_csv(a)
print(a)
print(type(a))

    keyword  count
0        잇몸   1547
1         고   1304
2        양제    923
3        건강    744
4        치아    700
..      ...    ...
354      냄새     20
355      권장     20
356      구성     20
357      결합     20
358       감     20

[359 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
